<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_GRAPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator

In [17]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.1.0


In [29]:
class Propagator:
    def __init__(self, no_of_steps, delta_t):
        self.delta_t = delta_t
        self.x = tf.constant(
            [[0, 1], [1, 0]], dtype=tf.complex128
        )
        self.y = tf.constant(
            [[0 + 0j, 0 - 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
        )

        self.generators =  tf.stack([self.x, self.y])
        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 2], dtype=tf.float64), dtype=tf.float64
        )

        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        regularized_amplitudes = 1 / np.sqrt(2) * tf.math.tanh(
            self.ctrl_amplitudes
        )

        exponents = tf.linalg.tensordot(
            tf.cast(regularized_amplitudes, dtype=tf.complex128),
            -2 * np.pi *(0 + 1j) * self.delta_t * self.generators, 1
        )
        return tf.linalg.expm(exponents)
    
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    @tf.function
    def infidelity(self):
        propagator = self.propagate()
        tr = tf.linalg.trace(tf.linalg.matmul(self.x, propagator))
        return 1 - tf.math.real(tr * tf.math.conj(tr)) / (2 ** 2)

propagator = Propagator(1000, 0.001)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitudes.assign(
    tf.random.uniform([1000, 2], -1, 1, dtype=tf.float64)
)

def optimization_step():
    with tf.GradientTape() as tape:
        infidelity = propagator.infidelity()
    gradients = tape.gradient(infidelity, [propagator.ctrl_amplitudes])
    optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitudes]))
    return infidelity

steps = range(100)
for step in steps:
    current_infidelity = optimization_step()
    print('step %2d: infidelity=%2.5f' %
          (step, current_infidelity))
    
propagator.ctrl_amplitudes.numpy()

step  0: infidelity=0.99892
step  1: infidelity=0.99556
step  2: infidelity=0.99012
step  3: infidelity=0.98260
step  4: infidelity=0.97296
step  5: infidelity=0.96119
step  6: infidelity=0.94725
step  7: infidelity=0.93113
step  8: infidelity=0.91282
step  9: infidelity=0.89234
step 10: infidelity=0.86971
step 11: infidelity=0.84499
step 12: infidelity=0.81826
step 13: infidelity=0.78961
step 14: infidelity=0.75916
step 15: infidelity=0.72703
step 16: infidelity=0.69337
step 17: infidelity=0.65837
step 18: infidelity=0.62225
step 19: infidelity=0.58521
step 20: infidelity=0.54748
step 21: infidelity=0.50932
step 22: infidelity=0.47102
step 23: infidelity=0.43284
step 24: infidelity=0.39506
step 25: infidelity=0.35796
step 26: infidelity=0.32183
step 27: infidelity=0.28692
step 28: infidelity=0.25349
step 29: infidelity=0.22175
step 30: infidelity=0.19192
step 31: infidelity=0.16415
step 32: infidelity=0.13859
step 33: infidelity=0.11533
step 34: infidelity=0.09442
step 35: infidelity=

array([[ 0.31967052,  0.12724451],
       [ 1.25829074, -0.33613855],
       [-0.13067111,  0.21519696],
       ...,
       [ 0.74399155, -0.0846596 ],
       [-0.22153783, -0.21390806],
       [ 0.75540621, -1.01513137]])